In [31]:
'''
获取手势mark数据与label, 保存到xml/json 用于动态手势训练
'''


'\n获取手势mark数据与label, 保存到xml/json 用于动态手势训练\n'

In [32]:
import cv2
import json
import mediapipe as mp
import os
import pandas as pd
from PIL import Image
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
import random
import matplotlib.pyplot as plt

In [33]:


cfile = '/Users/mac/vscode/python/ML/data/all_pic/'
file_list = os.listdir(cfile)
df = pd.DataFrame(file_list,columns=['name'])

df_0 = df[df.name.str.contains('_0')]
df_1 = df[df.name.str.contains('_1')]
df_2 = df[df.name.str.contains('_2')]
df_5 = df[df.name.str.contains('_5')]
other_label = ['_3', '_4', '_6', '_7', '_8', '_9']
other_labels = '|'.join(other_label)
df_other = df[df.name.str.contains(other_labels)]


In [35]:
len(df_other)

1109

In [36]:
'''
数据增强处理 数据归一化处理

所有mark都 减去 mark[0]

'''
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.7)

def mirror(image):
    img = cv2.flip(image,2)
    return img

def rotate(image):
    rows, cols = image.shape[:2]
    r = random.random()*90 - 45
    s = random.random()*0.15 + 0.9
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), r, s)#中心 角度 缩放s
    dst = cv2.warpAffine(image, M, (cols, rows))
    return dst


#正则化
def get_float(s):
    return (float(s.split(':')[-1]))
def regularization(mark_list):
    xyz_ = []
    x0 = get_float(mark_list[0][0])
    y0 = get_float(mark_list[0][1])
    z0 = get_float(mark_list[0][2])
    for i, m in enumerate(mark_list):
        x = get_float(m[0]) - x0
        y = get_float(m[1]) - y0
        z = get_float(m[2]) - z0
        xyz_.append([x,y,z])
    return ([(x0), (y0), (z0)], xyz_)

# 获取mark数据
def take_mark_data(image):
    results = hands.process(image)
    if results.multi_hand_landmarks:
        marks = str(results.multi_hand_landmarks[0])
        marks = marks.replace(' ','').replace('\n',' ').split('landmark')
        marks.pop(0)
        mark_list=[]
        for i in marks:
            mark_list.append(i.split(' ')[1:6])
        xyz0, xyz_ = regularization(mark_list)
        xyz_[0] = xyz0
        return (xyz_)
        # 画图
        # annotated_image = image.copy()
        # for hand_landmarks in results.multi_hand_landmarks:
        #     #print('hand_landmarks:', hand_landmarks)
        #     mp_drawing.draw_landmarks(
        #         annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        # im = Image.fromarray(annotated_image)
        # plt.imshow(im)
    else:
        
        return False



In [41]:
path = "../data/mark.json"
with open(path) as f:
    data = json.load(f)

In [42]:
len(data['2'])

257

In [43]:
print(len(df_0),len(df_1),len(df_2),len(df_5))

188 261 234 186


In [48]:
'''
To do!
'''
def add_data(num,label:str,df_):
    get_mark_ = df_.name.sample(num)
    label = label

    for n in get_mark_:

        file = cfile + '/' + n
        #label = n.split('_')[-1].split('.')[0]
        image = cv2.flip(cv2.cvtColor(cv2.imread(file), cv2.COLOR_BGR2RGB), 1) #垂直翻转 转颜色通道
        if random.random()>0.5:
            image = mirror(image)
        image = rotate(image)

        xyz  = take_mark_data(image)

        if label not in data:
            data.setdefault(label,[])
            
        if xyz:
            data[label].append(xyz)

    print(len(data[label]))

In [50]:
add_data(180, '0', df_1)
add_data(180, '1', df_1)
add_data(180, '2', df_1)
add_data(180, '5', df_1)

484
499
542
488


In [51]:
'''
写入数据信息
'''
with open(path, "w+") as f:
    json.dump(data,f)
